In [1]:
# !kaggle competitions download -c playground-series-s5e3
# !unzip -u *.zip

In [2]:
from pathlib import Path
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import missingno

from sklearn import set_config
set_config(transform_output = "pandas")

from sklearn.model_selection import ShuffleSplit, KFold, StratifiedKFold
from sklearn.model_selection import cross_validate, GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

KAGGLE_RUN = False
if KAGGLE_RUN:
    working_dir = Path('/kaggle/input/playground-series-s5e3')
else:
    working_dir = Path().cwd()

In [3]:
train_df = pd.read_csv(working_dir/'train.csv', index_col='id')
test_df = pd.read_csv(working_dir/'test.csv', index_col='id')


In [4]:
train_df

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2185,361,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,22.1,1
2186,362,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,35.3,1
2187,363,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,32.9,1


In [5]:
test_df

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4
...,...,...,...,...,...,...,...,...,...,...,...
2915,361,1020.8,18.2,17.6,16.1,13.7,96.0,95.0,0.0,20.0,34.3
2916,362,1011.7,23.2,18.1,16.0,16.0,78.0,80.0,1.6,40.0,25.2
2917,363,1022.7,21.0,18.5,17.0,15.5,92.0,96.0,0.0,50.0,21.9


In [6]:
NUMERIC_COLUMNS=['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']
CATEGORIC_COLUMNS=[]
TARGET_COLUMN=['rainfall']
ALL_COLUMNS=NUMERIC_COLUMNS+CATEGORIC_COLUMNS+TARGET_COLUMN

In [7]:
# feature engineering
#  add lag, fourier features, spreads, binning, days with maxtemp< temaparature, etc.

def group_agg_merge(df, by_column, agg_column):
    grouped_df = df.groupby(by=by_column).agg(
        **{
        f'{by_column}_average_{agg_column}':(agg_column, 'mean'),
        f'{by_column}_std_{agg_column}':(agg_column, 'std'),
        f'{by_column}_skew_{agg_column}':(agg_column, 'skew'),
        f'{by_column}_median_{agg_column}':(agg_column, 'median'),
        f'{by_column}_min_{agg_column}':(agg_column, 'min'),
        f'{by_column}_max_{agg_column}':(agg_column, 'max'),}
    )
    df = df.join(grouped_df, on=by_column, how='left')
    return df

def add_features(df):
    df=df.fillna(0)
    df['temp_spread'] = df['maxtemp'] - df['mintemp']
    df['temp_diff_dew'] = df['temparature'] - df['dewpoint']
    df['pressure_bins'] = pd.cut(df['pressure'], bins=[998, 1010, 1020, 1035], labels=list(range(3)), include_lowest=True)
    df['humidity_bins'] = pd.cut(df['humidity'], bins=[0, 50, 80,100], labels=list(range(3)), include_lowest=True)
    df['cloud_bins'] = pd.cut(df['cloud'], bins=[0, 25, 50, 75, 100], labels=list(range(4)), include_lowest=True)
    df['sunshine_bins'] = pd.cut(df['sunshine'], bins=[-0.1, 3., 6., 9., 13.], labels=list(range(4)), include_lowest=True)
    df['winddirection_bins'] = pd.cut(df['winddirection'], bins=[-1., 90, 180, 270, 361.], labels=list(range(4)), include_lowest=True)
    df['windspeed_bins'] = pd.cut(df['windspeed'], bins=[0, 15, 30, 45, 60], labels=list(range(4)), include_lowest=True)

    df['max_temp_under_temp'] = df['maxtemp'] < df['temparature']
    df['min_temp_over_temp'] = df['mintemp'] > df['temparature']
    
    df['month'] = df['day'].apply(
        lambda x: 'Jan' if 1<=x<=31 
        else 'Feb' if 32<= x<= 59
        else 'Mar' if 60<= x<= 90
        else 'Apr' if 91<= x<= 120
        else 'May' if 121<= x<= 151
        else 'Jun' if 152<= x<= 181
        else 'Jul' if 182<= x<= 212
        else 'Aug' if 213<= x<= 243
        else 'Sep' if 244<= x<= 273
        else 'Oct' if 274<= x<= 304
        else 'Nov' if 305<= x<= 334
        else 'Dec' if 335<= x<= 366
        else 'Nan'
    )

    df['season'] = df['month'].apply(
        lambda x: 'Winter' if x in ['Dec', 'Jan', 'Feb']
        else 'Spring' if x in ['Mar', 'Apr', 'May']
        else 'Summer' if x in ['Jun', 'Jul', 'Aug']
        else 'Autumn' if x in ['Sep', 'Oct', 'Nov']
        else 'Nan'
    )

    for i, k in itertools.product(['month', 'season'], ['temparature', 'pressure', 'dewpoint', 'humidity', 'sunshine', 'windspeed']):
        df = group_agg_merge(df, i, k)

    return df

NUMERIC_COLUMNS+=[
    'temp_spread',
    'temp_diff_dew',
    ]+[f'{i}_{j}_{k}' for i,j,k in itertools.product(['month', 'season'], ['average', 'std', 'skew', 'median', 'min', 'max'], ['temparature', 'pressure', 'dewpoint', 'humidity', 'sunshine', 'windspeed'])]
CATEGORIC_COLUMNS+=[
    'pressure_bins',
    'humidity_bins',
    'cloud_bins',
    'sunshine_bins',
    'winddirection_bins',
    'windspeed_bins',
    'month',
    'season',
    'max_temp_under_temp',
    'min_temp_over_temp'
    ]

train_df = add_features(train_df)
test_df = add_features(test_df)

In [8]:
print(NUMERIC_COLUMNS)
print(CATEGORIC_COLUMNS)

['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed', 'temp_spread', 'temp_diff_dew', 'month_average_temparature', 'month_average_pressure', 'month_average_dewpoint', 'month_average_humidity', 'month_average_sunshine', 'month_average_windspeed', 'month_std_temparature', 'month_std_pressure', 'month_std_dewpoint', 'month_std_humidity', 'month_std_sunshine', 'month_std_windspeed', 'month_skew_temparature', 'month_skew_pressure', 'month_skew_dewpoint', 'month_skew_humidity', 'month_skew_sunshine', 'month_skew_windspeed', 'month_median_temparature', 'month_median_pressure', 'month_median_dewpoint', 'month_median_humidity', 'month_median_sunshine', 'month_median_windspeed', 'month_min_temparature', 'month_min_pressure', 'month_min_dewpoint', 'month_min_humidity', 'month_min_sunshine', 'month_min_windspeed', 'month_max_temparature', 'month_max_pressure', 'month_max_dewpoint', 'month_max_humidity', 'month_max_sunshi

In [9]:
target = train_df[TARGET_COLUMN]
train = train_df.drop(columns=TARGET_COLUMN)
test = test_df

In [10]:
target

,rainfall
id,
0,1
1,1
2,1
3,1
4,0
...,...
2185,1
2186,1
2187,1


In [11]:
train

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,...,season_skew_sunshine,season_median_sunshine,season_min_sunshine,season_max_sunshine,season_average_windspeed,season_std_windspeed,season_skew_windspeed,season_median_windspeed,season_min_windspeed,season_max_windspeed
id,,,,,,,,,,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,361,1014.6,23.2,20.6,19.1,19.9,97.0,88.0,0.1,40.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
2186,362,1012.4,17.2,17.3,16.3,15.3,91.0,88.0,0.0,50.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5
2187,363,1013.3,19.0,16.3,14.3,12.6,79.0,79.0,5.0,40.0,...,1.452287,0.3,0.0,11.2,26.444465,10.318128,0.560474,25.1,6.9,59.5


In [12]:
test

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,...,season_skew_sunshine,season_median_sunshine,season_min_sunshine,season_max_sunshine,season_average_windspeed,season_std_windspeed,season_skew_windspeed,season_median_windspeed,season_min_windspeed,season_max_windspeed
id,,,,,,,,,,,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,361,1020.8,18.2,17.6,16.1,13.7,96.0,95.0,0.0,20.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2916,362,1011.7,23.2,18.1,16.0,16.0,78.0,80.0,1.6,40.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5
2917,363,1022.7,21.0,18.5,17.0,15.5,92.0,96.0,0.0,50.0,...,1.271779,0.15,0.0,10.5,26.370556,10.223588,0.631526,25.15,8.7,59.5


In [13]:
transformer = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), NUMERIC_COLUMNS),
        ('categories', OneHotEncoder(sparse_output=False), CATEGORIC_COLUMNS),
    ], remainder='passthrough'
)

classifier = CatBoostClassifier(verbose=0)

pipe = Pipeline(
    steps=[
        ('transform_columns', transformer),
        ('classifier', classifier)
        ]
        )


In [14]:
cv_results = cross_validate(
    pipe,
    train,
    target,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=13),
    scoring="roc_auc",
    n_jobs=2
)

errors_tree_regressor = pd.Series(
    cv_results["test_score"]
)
errors_tree_regressor.describe()

count    5.000000
mean     0.888923
std      0.011092
min      0.875140
25%      0.881734
50%      0.887682
75%      0.898962
max      0.901094
dtype: float64

In [15]:

cv_search = GridSearchCV(
    estimator = pipe,
    param_grid={
        # 'classifier__n_estimators':[100, 500],
        # 'classifier__n_estimators':[10, 50, 100, 500],
        # 'classifier__max_depth':[6, 10, 50, 100],
        # 'classifier__max_leaves':[0, 5, 10],
        'classifier__learning_rate':[0.1, 0.3, 0.5],
        # 'classifier__subsample':[0.8, 0.9, 1],
        # 'classifier__colsample_bytree':[0.8, 0.9, 1],
        # 'classifier__criterion':['squared_error', 'friedman_mse', 'poisson'],
    },
    scoring="roc_auc",
    n_jobs=3,
)

search_results = cv_search.fit(
    train,
    target
)


In [16]:
pd.DataFrame(cv_search.cv_results_).T

,0,1,2
mean_fit_time,9.489742,8.71108,7.342302
std_fit_time,1.829348,1.17634,0.405722
mean_score_time,0.029083,0.042666,0.030908
std_score_time,0.007255,0.007849,0.006329
param_classifier__learning_rate,0.1,0.3,0.5
params,{'classifier__learning_rate': 0.1},{'classifier__learning_rate': 0.3},{'classifier__learning_rate': 0.5}
split0_test_score,0.84885,0.843238,0.849832
split1_test_score,0.90115,0.895763,0.873401
split2_test_score,0.886756,0.881958,0.86821
split3_test_score,0.856706,0.854938,0.846633


In [17]:
cv_search.best_score_

0.8706397306397307

In [18]:
cv_search.best_params_

{'classifier__learning_rate': 0.1}

In [19]:
sub_df = pd.DataFrame(
    index=test.index,
    data={
        'rainfall':cv_search.predict(test)
    },
)
sub_df    


,rainfall
id,
2190,1
2191,1
2192,1
2193,0
2194,0
...,...
2915,1
2916,1
2917,1


In [20]:
if KAGGLE_RUN:
    sub_df.to_csv("/kaggle/working/submission.csv", index_label='id')
    !head /kaggle/working/submission.csv